In [2]:
import numpy as np
import pandas as pd
import time
import datetime
import sys
import os
from tqdm import tqdm
from functools import reduce
from Bio import SeqIO
import joblib
sys.path.append("../tools/")
import funclib

sys.path.append("../")
import benchmark_train as btrain
import benchmark_test as btest
import benchmark_common as bcommon
import config as cfg
import benchmark_evaluation as eva
import embedding_esm as esmebd

import production as pdc

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from pandarallel import pandarallel #  import pandaralle
pandarallel.initialize() # init

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [64]:
input_file = cfg.DATADIR + 'test1.fasta'
output_file = cfg.RESULTSDIR + 'ec_res.tsv'
mode = 'r'
topk = 10

In [111]:
ecp = pdc.step_by_step_run(input_fasta=cfg.DATADIR + 'test1.fasta', 
                    output_tsv=cfg.RESULTSDIR + 'ecp_res.tsv', 
                    mode='p', topnum=1
                )

run in annoation mode
step 1: loading data
step 2: find existing data
step 3: Embedding
Transferred model to GPU


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 19.71it/s]


step 4: sequence alignment
Write finished


diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 80
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: /home/shizhenkun/codebase/DMLF/data/production_blast.fasta
Opening the database file...  [0.05s]
Loading sequences...  [0.991s]
Masking sequences...  [0.35s]
Writing sequences...  [0.184s]
Hashing sequences...  [0.057s]
Loading sequences...  [0s]
Writing trailer...  [0.003s]
Closing the input file...  [0.002s]
Closing the database file...  [0.201s]
Database hash = d47ea1d106d79f6ffaaaf02f417a2936
Processed 565254 sequences, 203850821 letters.
Total time = 1.842s


Write finished
diamond blastp -d /home/shizhenkun/codebase/DMLF/data/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet
step 5: predict isEnzyme
step 6: predict function counts
step 7: predict EC
slice files prepared success
./slice_predict /home/shizhenkun/codebase/DMLF/tmp/ptest_2021_11_15_08_07_11.txt /home/shizhenkun/codebase/DMLF/model/slice_esm32 /home/shizhenkun/codebase/DMLF/tmp/ptest_2021_11_15_08_07_11.tsv -o 32 -b 0 -t 32 -q 0


Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


Time taken to find approx nearest neighbors = 0.004502
Total prediction time: 0.580303 s
Prediction time per point: 41.4502 ms
step 8: integrate results
step 9: writting results
All done running time: 140.57902397299995 Seconds


In [112]:
ecp

In [115]:
ecr = pdc.step_by_step_run(input_fasta=cfg.DATADIR + 'test1.fasta',  output_tsv=cfg.RESULTSDIR + 'ecr_res.tsv',  mode='r', topnum=5 )

run in recommendation mode
step 1: loading data
step 2: find existing data
step 3: Embedding
Transferred model to GPU


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 464/464 [00:24<00:00, 19.13it/s]


step 4: sequence alignment
Write finished


diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 80
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: /home/shizhenkun/codebase/DMLF/data/production_blast.fasta
Opening the database file...  [0.067s]
Loading sequences...  [0.979s]
Masking sequences...  [0.382s]
Writing sequences...  [0.185s]
Hashing sequences...  [0.058s]
Loading sequences...  [0s]
Writing trailer...  [0.003s]
Closing the input file...  [0.002s]
Closing the database file...  [0.206s]
Database hash = d47ea1d106d79f6ffaaaf02f417a2936
Processed 565254 sequences, 203850821 letters.
Total time = 1.885s


Write finished
diamond blastp -d /home/shizhenkun/codebase/DMLF/data/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet
step 5: predict isEnzyme
step 6: predict function counts
step 7: predict EC
slice files prepared success
./slice_predict /home/shizhenkun/codebase/DMLF/tmp/ptest_2021_11_15_08_19_27.txt /home/shizhenkun/codebase/DMLF/model/slice_esm32 /home/shizhenkun/codebase/DMLF/tmp/ptest_2021_11_15_08_19_27.tsv -o 32 -b 0 -t 32 -q 0


Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


Time taken to find approx nearest neighbors = 0.062284
Total prediction time: 0.751984 s
Prediction time per point: 1.62066 ms
step 8: integrate results
step 9: writting results
All done running time: 862.5524471500003 Seconds


In [19]:
start = time.process_time()
if mode =='p':
    print('run in annoation mode')
if mode =='r':
    print('run in annoation mode')

# 1. 读入数据
print('step 1: loading data')
input_df = funclib.load_fasta_to_table(input_file) # test fasta
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER) #sprot db

# 2. 查找数据
print('step 2: find existing data')
find_data =input_df.merge(latest_sprot, on='seq', how='left')
exist_data= find_data[~find_data.name.isnull()].iloc[:,np.r_[0,2,1,12,7,9:12]].rename(columns={'id_x':'id','id_y':'id_uniprot'})
noExist_data = find_data[find_data.name.isnull()]
noExist_data.reset_index(drop=True, inplace=True)
noExist_data = noExist_data.iloc[:,np.r_[0,2,1,12,7,9:12]].rename(columns={'id_x':'id','id_y':'id_uniprot'})

if len(noExist_data) == 0:
    exist_data.to_csv(output_tsv, sep='\t')
    end = time.process_time()
    print('All done running time: %s Seconds'%(end-start))
    

# 3. EMBedding
print('step 3: Embedding')
if mode =='p':
    rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=noExist_data, model='esm1b_t33_650M_UR50S',seqthres=1022)
if mode == 'r':
    rep0, rep32, rep33 = esmebd.get_rep_multi_sequence(sequences=input_df, model='esm1b_t33_650M_UR50S',seqthres=1022)

# 4. sequence alignment
print('step 4: sequence alignment')
if ~os.path.exists(cfg.FILE_BLAST_PRODUCTION_DB):
    funclib.table2fasta(latest_sprot, cfg.FILE_BLAST_PRODUCTION_FASTA)
    cmd = r'diamond makedb --in {0} -d {1}'.format(cfg.FILE_BLAST_PRODUCTION_FASTA, cfg.FILE_BLAST_PRODUCTION_DB)
    os.system(cmd)
if mode =='p':
    blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=noExist_data)
if mode == 'r':
    blast_res = funclib.getblast_usedb(db=cfg.FILE_BLAST_PRODUCTION_DB, test=input_df)
blast_res = blast_res[['id', 'sseqid']].merge(latest_sprot, left_on='sseqid', right_on='id', how='left').iloc[:,np.r_[0,2:14]]
blast_res = blast_res.iloc[:,np.r_[0,1,11,12,6,8:11]].rename(columns={'id_x':'id','id_y':'id_uniprot'})

# 5. isEnzyme Prediction
print('step 5: predict isEnzyme')
model_isEnzyme = joblib.load(cfg.ISENZYME_MODEL)
pred_isEnzyme = pd.DataFrame()
pred_isEnzyme['id']=rep32.id
pred_isEnzyme['isEnzyme_pred'] = model_isEnzyme.predict(rep32.iloc[:,1:])

# 6. How many Prediction
print('step 6: predict function counts')
pred_howmany = pdc.predict_function_counts(rep32)

# 7. EC Prediction
print('step 7: predict EC')
pred_ec = pdc.predict_ec_slice(test_data=rep32)
if mode=='p':
    pred_ec = noExist_data[['id','seq']].merge(pred_ec, on='id', how='left')
if mode == 'r':
    pred_ec = input_df[['id', 'seq']].merge(pred_ec, on='id', how='left')
pred_ec['seqlength']=pred_ec.seq.parallel_apply(lambda x: len(x) )

run in annoation mode
step 1: loading data
step 2: find existing data
step 3: Embedding
Transferred model to GPU


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 464/464 [00:23<00:00, 19.41it/s]


step 4: sequence alignment
Write finished


diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 80
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: /home/shizhenkun/codebase/DMLF/data/production_blast.fasta
Opening the database file...  [0.035s]
Loading sequences...  [1.003s]
Masking sequences...  [0.442s]
Writing sequences...  [0.185s]
Hashing sequences...  [0.058s]
Loading sequences...  [0s]
Writing trailer...  [0.003s]
Closing the input file...  [0.002s]
Closing the database file...  [0.22s]
Database hash = d47ea1d106d79f6ffaaaf02f417a2936
Processed 565254 sequences, 203850821 letters.
Total time = 1.952s


Write finished
diamond blastp -d /home/shizhenkun/codebase/DMLF/data/production_blast.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1 --quiet
step 5: predict isEnzyme
step 6: predict function counts
step 7: predict EC
slice files prepared success
./slice_predict /home/shizhenkun/codebase/DMLF/tmp/ptest_2021_11_15_07_00_22.txt /home/shizhenkun/codebase/DMLF/model/slice_esm32 /home/shizhenkun/codebase/DMLF/tmp/ptest_2021_11_15_07_00_22.tsv -o 32 -b 0 -t 32 -q 0


Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


Time taken to find approx nearest neighbors = 0.062579
Total prediction time: 0.562644 s
Prediction time per point: 1.21259 ms


In [63]:
existing_table=exist_data
blast_table=blast_res
isEnzyme_pred_table = pred_isEnzyme
how_many_table = pred_howmany
ec_table = pred_ec
topnum=topk

In [109]:
existing_table['res_type'] = 'db_match'
blast_table['res_type']='blast_match'
results_df = ec_table.merge(blast_table, on='id', how='left')

function_df = how_many_table.copy()
function_df = function_df.merge(isEnzyme_pred_table, on='id', how='left')
function_df = function_df.merge(blast_table[['id', 'ec_number']], on='id', how='left')
function_df['pred_function_counts']=function_df.parallel_apply(lambda x : pdc.integrate_enzyme_functioncounts(x.ec_number, x.isEnzyme_pred, x.pred_s, x.pred_m), axis=1)
results_df = results_df.merge(function_df[['id','pred_function_counts']],on='id',how='left')

results_df.loc[results_df[results_df.res_type.isnull()].index,'res_type']='dmlf_pred'
results_df['pred_ec']=results_df.parallel_apply(lambda x: pdc.gather_ec_by_fc(x.iloc[3:23],x.ec_number, x.pred_function_counts), axis=1)
results_df = results_df.iloc[:,np.r_[0,23,1,2,32,27:31]].rename(columns={'seq_x':'seq','seqlength_x':'seqlength'})



if mode=='p':
    existing_table['pred_ec']=''
    result_set = pd.concat([existing_table, results_df], axis=0)
    result_set = result_set.drop_duplicates(subset=['id'], keep='first').sort_values(by='res_type')
    result_set['ec_number'] = result_set.apply(lambda x: x.pred_ec if str(x.ec_number)=='nan' else x.ec_number, axis=1)
    result_set.reset_index(drop=True, inplace=True)
    result_set = result_set.iloc[:,0:9]
    
if mode =='r':
    result_set= results_df.merge(ec_table, on=['id'], how='left')
    result_set=result_set.iloc[:,np.r_[0:3,30,5:9, 4,10:30]]
    result_set = result_set.rename(columns=dict({'seq_x': 'seq','pred_ec': 'top0','top0_y': 'top1' },  **{'top'+str(i) : 'top'+str(i+1) for i in range(0, 20)}))
    result_set = result_set.iloc[:,0:(8+topk)]

In [110]:
result_set

,id,id_uniprot,seq,seqlength,date_integraged,date_sequence_update,date_annotation_update,res_type,top0,top1,top2,top3,top4,top5,top6,top7,top8,top9
0,t1,P69031,MAFLKRRDLISIKSLFLVLFLGLVSLSICEQEKREEENQEEDEENE...,78,01-FEB-2005,17-JUN-2020,02-JUN-2021,blast_match,-,3.4.24.-,3.1.27.-,3.4.22.-,3.4.21.-,1.6.5.-,3.1.27.5,2.3.1.251,2.1.1.86,2.3.2.27
1,t2,P69019,MAFLKKSLFLVLFLGKKKKKKKKKKKLVSLSICEKEKRQNEEDEDE...,83,01-FEB-2005,17-JUN-2020,02-JUN-2021,blast_match,-,3.1.1.4,2.1.1.-,2.4.1.-,4.1.1.39,3.4.21.-,3.4.22.-,3.4.-.-,3.1.4.35,2.4.-.-
2,t3,Q5RF96,MARGGDTGCTGPSETSASMMMMMMMMMGAVAIAFPGLEGPPADAQY...,178,30-AUG-2005,16-OCT-2019,02-JUN-2021,blast_match,3.4.-.-,3.4.-.-,3.4.23.-,3.1.2.-,2.4.99.18,4.2.99.22,3.2.1.14,2.3.2.27,3.4.21.98,3.1.21.-
3,t4,NaN,KERKTGMFVMHEIGTPMEVRESGCDDKHHPHLKMMRYDEFADAILW...,60,NaN,NaN,NaN,dmlf_pred,-,3.1.27.-,1.4.3.2,4.1.2.13,2.1.1.148,3.6.4.-,1.1.1.72,2.7.1.19,3.4.22.45,1.5.98.1
4,r1,NaN,HLYCMFAQQATADYGGGNQYSYPYMPPGCWLPWLYNQCKQDHDDCN...,500,NaN,NaN,NaN,dmlf_pred,-,3.2.1.18,2.7.7.48,3.4.22.-,3.1.27.-,2.1.1.56,2.7.7.88,2.8.4.1,2.1.1.296,3.4.22.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,F4KH86,F4KH86,MDSDAWEIIHIPEKPSLSPDHQPTVKVYASLIKPRFANTIVRHLCK...,400,25-APR-2018,28-JUN-2011,02-JUN-2021,blast_match,3.5.4.33,3.5.4.5,3.5.4.33,3.5.4.-,2.3.2.27,3.5.4.12,2.3.1.-,3.1.26.4,4.2.1.109,2.8.1.12
460,Q81LM1,Q81LM1,MISVNKVFYAHSERFQMQNMNVHIKAGEVVSLIGPNGSGKSTLLRL...,272,25-APR-2018,01-JUN-2003,02-JUN-2021,blast_match,7.2.2.-,3.6.3.-,3.6.3.28,3.6.3.33,3.6.3.41,3.6.3.34,3.6.3.24,2.3.1.181,2.3.1.128,1.3.99.-
461,Q9LS80,Q9LS80,MVRNQSRCFLGIIITISFFFFLLSLPNTFASPTRSLCRSDQRDALL...,835,25-APR-2018,01-OCT-2000,07-APR-2021,blast_match,-,2.7.11.1,3.1.1.11,3.2.1.67,2.7.10.1,3.4.24.-,6.1.1.10,2.7.11.-,4.1.99.22,3.4.16.-
462,I1RW37,I1RW37,MHQQPRGPARVSSPGATTQPNLPSRSNSTREAALGSRPRAGSNLVG...,927,25-APR-2018,13-JUN-2012,02-JUN-2021,blast_match,-,2.7.11.1,2.3.2.27,2.7.11.25,3.4.-.-,2.3.2.-,3.1.3.48,1.-.-.-,3.4.21.97,2.1.1.17


In [105]:
topk

10

In [60]:
results_df

,id,seq_x,top0,top1,top2,top3,top4,top5,top6,top7,...,seqlength_x,id_uniprot,seq_y,seqlength_y,ec_number,date_integraged,date_sequence_update,date_annotation_update,res_type,pred_function_counts
0,t1,MAFLKRRDLISIKSLFLVLFLGLVSLSICEQEKREEENQEEDEENE...,3.4.24.-,3.1.27.-,3.4.22.-,3.4.21.-,1.6.5.-,3.1.27.5,2.3.1.251,2.1.1.86,...,78,P69031,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...,71.0,-,01-FEB-2005,17-JUN-2020,02-JUN-2021,blast_match,0
1,t2,MAFLKKSLFLVLFLGKKKKKKKKKKKLVSLSICEKEKRQNEEDEDE...,3.1.1.4,2.1.1.-,2.4.1.-,4.1.1.39,3.4.21.-,3.4.22.-,3.4.-.-,3.1.4.35,...,83,P69019,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...,72.0,-,01-FEB-2005,17-JUN-2020,02-JUN-2021,blast_match,0
2,t3,MARGGDTGCTGPSETSASMMMMMMMMMGAVAIAFPGLEGPPADAQY...,3.4.-.-,3.4.23.-,3.1.2.-,2.4.99.18,4.2.99.22,3.2.1.14,2.3.2.27,3.4.21.98,...,178,Q5RF96,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,169.0,3.4.-.-,30-AUG-2005,16-OCT-2019,02-JUN-2021,blast_match,1
3,t4,KERKTGMFVMHEIGTPMEVRESGCDDKHHPHLKMMRYDEFADAILW...,3.1.27.-,1.4.3.2,4.1.2.13,2.1.1.148,3.6.4.-,1.1.1.72,2.7.1.19,3.4.22.45,...,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,r1,HLYCMFAQQATADYGGGNQYSYPYMPPGCWLPWLYNQCKQDHDDCN...,3.2.1.18,2.7.7.48,3.4.22.-,3.1.27.-,2.1.1.56,2.7.7.88,2.8.4.1,2.1.1.296,...,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459,F4KH86,MDSDAWEIIHIPEKPSLSPDHQPTVKVYASLIKPRFANTIVRHLCK...,3.5.4.5,3.5.4.33,3.5.4.-,2.3.2.27,3.5.4.12,2.3.1.-,3.1.26.4,4.2.1.109,...,400,F4KH86,MDSDAWEIIHIPEKPSLSPDHQPTVKVYASLIKPRFANTIVRHLCK...,400.0,3.5.4.33,25-APR-2018,28-JUN-2011,02-JUN-2021,blast_match,1
460,Q81LM1,MISVNKVFYAHSERFQMQNMNVHIKAGEVVSLIGPNGSGKSTLLRL...,3.6.3.-,3.6.3.28,3.6.3.33,3.6.3.41,3.6.3.34,3.6.3.24,2.3.1.181,2.3.1.128,...,272,Q81LM1,MISVNKVFYAHSERFQMQNMNVHIKAGEVVSLIGPNGSGKSTLLRL...,272.0,7.2.2.-,25-APR-2018,01-JUN-2003,02-JUN-2021,blast_match,1
461,Q9LS80,MVRNQSRCFLGIIITISFFFFLLSLPNTFASPTRSLCRSDQRDALL...,2.7.11.1,3.1.1.11,3.2.1.67,2.7.10.1,3.4.24.-,6.1.1.10,2.7.11.-,4.1.99.22,...,835,Q9LS80,MVRNQSRCFLGIIITISFFFFLLSLPNTFASPTRSLCRSDQRDALL...,835.0,-,25-APR-2018,01-OCT-2000,07-APR-2021,blast_match,0
462,I1RW37,MHQQPRGPARVSSPGATTQPNLPSRSNSTREAALGSRPRAGSNLVG...,2.7.11.1,2.3.2.27,2.7.11.25,3.4.-.-,2.3.2.-,3.1.3.48,1.-.-.-,3.4.21.97,...,927,I1RW37,MHQQPRGPARVSSPGATTQPNLPSRSNSTREAALGSRPRAGSNLVG...,927.0,-,25-APR-2018,13-JUN-2012,02-JUN-2021,blast_match,0
